## Source: https://huggingface.co/blog/sentiment-analysis-python

## Note: A Couple Hiccups
0. Issue occurs where you have to 'Run All' twice due to 'accelerate' library not working, restart and run again

1. Need the data from my Google Drive below and upload the .csv to session storage in Colab
https://drive.google.com/file/d/1GtHhmDXqx9bcXVJvPmGuJFQSdpoYmWv1/view?usp=sharing

2. Need to use your accesss token for HuggingFace

3. Issue with the Trainer() cell? Try running again with 'Restart Session and Run All'



# 0. How to Use Pre-trained Sentiment Analysis Models with Python

In [ ]:
# !pip install torch
# !pip install pytorch-accelerated

#!pip install accelerate -U
#!pip install transformers -U

In [ ]:
# There are more than 215 sentiment analysis models publicly available on the Hub and integrating them with Python just takes 5 lines of code
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that aut

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [ ]:
# Use a specific sentiment analysis model that is better suited to your language or use case by providing the name of the model.
# For example, if you want a sentiment analysis model for tweets, you can specify the model id
specific_model = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")
specific_model(data)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


[{'label': 'POS', 'score': 0.9916695356369019},
 {'label': 'NEG', 'score': 0.9806600213050842}]

# 1. Activate GPU and Install Dependencies

In [ ]:
# Activate GPU for faster training by clicking on 'Runtime' > 'Change runtime type' and then selecting GPU as the Hardware accelerator
# Then check if GPU is available
import torch
torch.cuda.is_available()

True

In [ ]:
# Install required libraries
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


#2. Preprocess data

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving fin_data_labeled.csv to fin_data_labeled (1).csv


In [ ]:
# Load data
from datasets import Dataset, load_dataset
from google.colab import files
import pandas as pd

# Example IMDB Dataset
#imdb = load_dataset("imdb")
#print("IMDB\n", imdb)
#print(imdb['train'][0])

# GUI for File Upload
#uploaded = files.upload()

# Processing 'fin_data.csv' from Section 16.4
#df = pd.read_csv("fin_data.csv")
#print("MAX", df['News Headline'].str.len().max())
#df['Label'] = df['Sentiment'].replace({'negative': 0, 'neutral': 1, 'positive': 2})
#del df['Unnamed: 0']
#df.to_csv('fin_data.csv', index=False)
#print(df)


fin = load_dataset("csv", data_files="fin_data_labeled.csv", split="train")
fin = fin.rename_column("News Headline", "text")
fin = fin.rename_column("Label", "label")
print("FIN\n", fin)

fin = fin.train_test_split(test_size=0.2)
print("FIN Train Test\n", fin)

FIN
 Dataset({
    features: ['text', 'label'],
    num_rows: 26228
})
FIN Train Test
 DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 20982
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5246
    })
})


In [ ]:
# Create a smaller training dataset for faster training times
small_train_dataset = fin["train"].shuffle(seed=0).select([i for i in list(range(1200))])
small_test_dataset = fin["test"].shuffle(seed=0).select([i for i in list(range(300))])
print("Train", small_train_dataset[0])
print("Test", small_test_dataset[0])

Train {'text': 'nice chart ty for the share', 'label': 2}
Test {'text': 'TV18 Broadcast Q4 profit at Rs 95.47 crore', 'label': 1}


In [ ]:
# Set DistilBERT tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
print(tokenized_train[0])
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

{'text': 'nice chart ty for the share', 'label': 2, 'input_ids': [101, 3835, 3673, 5939, 2005, 1996, 3745, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
# Use data_collector to convert our samples to PyTorch tensors and concatenate them with the correct amount of padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 3. Training the model

In [ ]:
!pip install evaluate

In [ ]:
# Define DistilBERT as our base model:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Define the evaluation metrics
# Loss Functions - https://www.datacamp.com/tutorial/loss-function-in-machine-learning
# HuggingFace Loss Function Usage - https://stackoverflow.com/questions/71581197/what-is-the-loss-function-used-in-trainer-from-the-transformers-library-of-huggi
import numpy as np
#from datasets import load_metric
from evaluate import load

from sklearn.metrics import mean_squared_error


def compute_metrics(eval_pred):
    #load_accuracy = load_metric("accuracy")
    #load_f1 = load_metric("f1")
    load_accuracy = load("accuracy")
    load_f1 = load("f1")
    load_mse = load("mse")
    load_mae = load("mae")

    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    f1 = load_f1.compute(predictions=predictions, references=labels, average='micro')["f1"]
    mse = load_mse.compute(predictions=predictions, references=labels, squared=True)["mse"]
    mae = load_mae.compute(predictions=predictions, references=labels)["mae"]
    return {"accuracy": accuracy, "f1": f1, "mse": mse, "mae": mae}

In [ ]:
# Log in to your Hugging Face account
# Get your API token here https://huggingface.co/settings/token
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install accelerate -U
!pip install transformers -U

In [ ]:
# Define a new Trainer with all the objects we constructed so far

from torch import nn
from transformers import TrainingArguments, Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]).to(model.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

repo_name = "financial-sentiment-model-1500-samples"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# Train the model
trainer.train()

Step,Training Loss


TrainOutput(global_step=150, training_loss=0.7649010721842447, metrics={'train_runtime': 30.4977, 'train_samples_per_second': 78.695, 'train_steps_per_second': 4.918, 'total_flos': 30484603755648.0, 'train_loss': 0.7649010721842447, 'epoch': 2.0})

In [ ]:
# Compute the evaluation metrics
trainer.evaluate()

{'eval_loss': 0.6277402639389038,
 'eval_accuracy': 0.6633333333333333,
 'eval_f1': 0.6633333333333333,
 'eval_mse': 0.8266666666666667,
 'eval_mae': 0.5,
 'eval_runtime': 3.5416,
 'eval_samples_per_second': 84.707,
 'eval_steps_per_second': 5.365,
 'epoch': 2.0}

# 4. Analyzing new data with multiple models - Self Sentiment Model, Bert Model, Twitter Roberta Model, ProsusAI Finbert Model, Sigma's Financial Sentiment Analysis Model, and DistilRoberta Financial Sentiment Model

In [ ]:
# Upload the model to the Hub
trainer.push_to_hub()

events.out.tfevents.1719127761.9bdd476daf66.3973.0:   0%|          | 0.00/5.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1719127795.9bdd476daf66.3973.1:   0%|          | 0.00/551 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kevinwlip/financial-sentiment-model-1500-samples/commit/172edf6de42e561bb1fbd2c012c6dbe234aff005', commit_message='End of training', commit_description='', oid='172edf6de42e561bb1fbd2c012c6dbe234aff005', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# Run inferences with your new model using Pipeline - https://huggingface.co/kevinwlip/financial-sentiment-model-1500-samples
from transformers import pipeline

sentiment_model = pipeline(model="kevinwlip/financial-sentiment-model-1500-samples")

sentiment_model(["I love this movie", "This movie sucks!"])

sentiment_model(["Energy shares drag on Wall Street as crude prices fall", "S&P 500 headed for a 67% downturn?", "Foreign investors navigate turmoil in Chinese markets with new playbook", "Fed Liquidity Drain Spoils Virus-Surge-Inspired Stock Buying-Panic", "Earnings per share ( EPS ) amounted to a loss of EUR0 .05",
                 "Equity markets likely to shine as Rupee's depreciation slows", "Investors cheered the blockbuster November jobs report", "Suven Life Sciences Q1 net up 16.68% at Rs 34.74 cr", "Airlines in sweet spot; expect SpiceJet to see rerating on IndiGo listing", "Twitter stock price target raised to $36.50 from $34.50 at Wedbush",
                 "A company is based in the district of the city", "The company did not disclose the price of the acquisition", "The 5 Best Finance Stocks of the Decade", "WeWork names new executives, path to profitability by 2023", "Chipotle highlights fast drive-thru lanes"])

config.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'LABEL_2', 'score': 0.4874809980392456},
 {'label': 'LABEL_2', 'score': 0.531254231929779},
 {'label': 'LABEL_2', 'score': 0.49818864464759827},
 {'label': 'LABEL_2', 'score': 0.462830126285553},
 {'label': 'LABEL_2', 'score': 0.5087128281593323},
 {'label': 'LABEL_2', 'score': 0.5211644172668457},
 {'label': 'LABEL_1', 'score': 0.5775545835494995},
 {'label': 'LABEL_2', 'score': 0.7830463647842407},
 {'label': 'LABEL_1', 'score': 0.7667332291603088},
 {'label': 'LABEL_2', 'score': 0.7404908537864685},
 {'label': 'LABEL_1', 'score': 0.7898803353309631},
 {'label': 'LABEL_1', 'score': 0.8366621136665344},
 {'label': 'LABEL_1', 'score': 0.6174285411834717},
 {'label': 'LABEL_1', 'score': 0.7031949758529663},
 {'label': 'LABEL_1', 'score': 0.690377414226532}]

In [ ]:
# Try using the latest Bert Model - https://huggingface.co/google-bert/bert-base-uncased
from transformers import pipeline

bert_model = pipeline("sentiment-analysis", model="google-bert/bert-base-uncased")

bert_model(["I love this movie", "This movie sucks!"])

bert_model(["Energy shares drag on Wall Street as crude prices fall", "S&P 500 headed for a 67% downturn?", "Foreign investors navigate turmoil in Chinese markets with new playbook", "Fed Liquidity Drain Spoils Virus-Surge-Inspired Stock Buying-Panic", "Earnings per share ( EPS ) amounted to a loss of EUR0 .05",
                 "Equity markets likely to shine as Rupee's depreciation slows", "Investors cheered the blockbuster November jobs report", "Suven Life Sciences Q1 net up 16.68% at Rs 34.74 cr", "Airlines in sweet spot; expect SpiceJet to see rerating on IndiGo listing", "Twitter stock price target raised to $36.50 from $34.50 at Wedbush",
                 "A company is based in the district of the city", "The company did not disclose the price of the acquisition", "The 5 Best Finance Stocks of the Decade", "WeWork names new executives, path to profitability by 2023", "Chipotle highlights fast drive-thru lanes"])

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[{'label': 'LABEL_0', 'score': 0.5133516192436218},
 {'label': 'LABEL_0', 'score': 0.5102410316467285},
 {'label': 'LABEL_0', 'score': 0.5599414110183716},
 {'label': 'LABEL_0', 'score': 0.5019082427024841},
 {'label': 'LABEL_0', 'score': 0.5638871192932129},
 {'label': 'LABEL_0', 'score': 0.551092803478241},
 {'label': 'LABEL_0', 'score': 0.539638876914978},
 {'label': 'LABEL_0', 'score': 0.5098826885223389},
 {'label': 'LABEL_0', 'score': 0.5092595219612122},
 {'label': 'LABEL_0', 'score': 0.5763128399848938},
 {'label': 'LABEL_0', 'score': 0.5446516871452332},
 {'label': 'LABEL_0', 'score': 0.5367032885551453},
 {'label': 'LABEL_0', 'score': 0.5440858602523804},
 {'label': 'LABEL_0', 'score': 0.5356974601745605},
 {'label': 'LABEL_0', 'score': 0.5273992419242859}]

In [ ]:
# Try using the latest Twitter Roberta Model - https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment-latest
from transformers import pipeline

roberta_model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

roberta_model(["I love this movie", "This movie sucks!"])

roberta_model(["Energy shares drag on Wall Street as crude prices fall", "S&P 500 headed for a 67% downturn?", "Foreign investors navigate turmoil in Chinese markets with new playbook", "Fed Liquidity Drain Spoils Virus-Surge-Inspired Stock Buying-Panic", "Earnings per share ( EPS ) amounted to a loss of EUR0 .05",
                 "Equity markets likely to shine as Rupee's depreciation slows", "Investors cheered the blockbuster November jobs report", "Suven Life Sciences Q1 net up 16.68% at Rs 34.74 cr", "Airlines in sweet spot; expect SpiceJet to see rerating on IndiGo listing", "Twitter stock price target raised to $36.50 from $34.50 at Wedbush",
                 "A company is based in the district of the city", "The company did not disclose the price of the acquisition", "The 5 Best Finance Stocks of the Decade", "WeWork names new executives, path to profitability by 2023", "Chipotle highlights fast drive-thru lanes"])

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

[{'label': 'negative', 'score': 0.5215955376625061},
 {'label': 'negative', 'score': 0.6893200874328613},
 {'label': 'neutral', 'score': 0.62848961353302},
 {'label': 'negative', 'score': 0.8247570991516113},
 {'label': 'negative', 'score': 0.5967459082603455},
 {'label': 'positive', 'score': 0.7096945643424988},
 {'label': 'positive', 'score': 0.8983082175254822},
 {'label': 'positive', 'score': 0.5159180164337158},
 {'label': 'positive', 'score': 0.7258645296096802},
 {'label': 'neutral', 'score': 0.6438215970993042},
 {'label': 'neutral', 'score': 0.9298118352890015},
 {'label': 'neutral', 'score': 0.8773128986358643},
 {'label': 'positive', 'score': 0.8506605625152588},
 {'label': 'positive', 'score': 0.7193376421928406},
 {'label': 'neutral', 'score': 0.5253716707229614}]

In [ ]:
# Try using the latest ProsusAI Finbert Model - https://huggingface.co/ProsusAI/finbert
from transformers import pipeline

finbert_model = pipeline("sentiment-analysis", model="ProsusAI/finbert")

finbert_model(["I love this movie", "This movie sucks!"])

finbert_model(["Energy shares drag on Wall Street as crude prices fall", "S&P 500 headed for a 67% downturn?", "Foreign investors navigate turmoil in Chinese markets with new playbook", "Fed Liquidity Drain Spoils Virus-Surge-Inspired Stock Buying-Panic", "Earnings per share ( EPS ) amounted to a loss of EUR0 .05",
                 "Equity markets likely to shine as Rupee's depreciation slows", "Investors cheered the blockbuster November jobs report", "Suven Life Sciences Q1 net up 16.68% at Rs 34.74 cr", "Airlines in sweet spot; expect SpiceJet to see rerating on IndiGo listing", "Twitter stock price target raised to $36.50 from $34.50 at Wedbush",
                 "A company is based in the district of the city", "The company did not disclose the price of the acquisition", "The 5 Best Finance Stocks of the Decade", "WeWork names new executives, path to profitability by 2023", "Chipotle highlights fast drive-thru lanes"])

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'label': 'negative', 'score': 0.9179688692092896},
 {'label': 'negative', 'score': 0.9538723230361938},
 {'label': 'neutral', 'score': 0.8057687878608704},
 {'label': 'negative', 'score': 0.50923752784729},
 {'label': 'negative', 'score': 0.7875516414642334},
 {'label': 'negative', 'score': 0.5851660370826721},
 {'label': 'positive', 'score': 0.6592963933944702},
 {'label': 'positive', 'score': 0.9493573307991028},
 {'label': 'positive', 'score': 0.8135269284248352},
 {'label': 'positive', 'score': 0.9495117664337158},
 {'label': 'neutral', 'score': 0.9443269371986389},
 {'label': 'neutral', 'score': 0.9468695521354675},
 {'label': 'neutral', 'score': 0.6568444967269897},
 {'label': 'neutral', 'score': 0.7671142220497131},
 {'label': 'neutral', 'score': 0.90207839012146}]

In [ ]:
# Try using the latest Sigma's Financial Sentiment Analysis Model - https://huggingface.co/Sigma/financial-sentiment-analysis
from transformers import pipeline

sigmafsa_model = pipeline("sentiment-analysis", model="Sigma/financial-sentiment-analysis")

sigmafsa_model(["I love this movie", "This movie sucks!"])

sigmafsa_model(["Energy shares drag on Wall Street as crude prices fall", "S&P 500 headed for a 67% downturn?", "Foreign investors navigate turmoil in Chinese markets with new playbook", "Fed Liquidity Drain Spoils Virus-Surge-Inspired Stock Buying-Panic", "Earnings per share ( EPS ) amounted to a loss of EUR0 .05",
                 "Equity markets likely to shine as Rupee's depreciation slows", "Investors cheered the blockbuster November jobs report", "Suven Life Sciences Q1 net up 16.68% at Rs 34.74 cr", "Airlines in sweet spot; expect SpiceJet to see rerating on IndiGo listing", "Twitter stock price target raised to $36.50 from $34.50 at Wedbush",
                 "A company is based in the district of the city", "The company did not disclose the price of the acquisition", "The 5 Best Finance Stocks of the Decade", "WeWork names new executives, path to profitability by 2023", "Chipotle highlights fast drive-thru lanes"])

config.json:   0%|          | 0.00/854 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'label': 'LABEL_0', 'score': 0.9958165287971497},
 {'label': 'LABEL_0', 'score': 0.9943016767501831},
 {'label': 'LABEL_0', 'score': 0.9543273448944092},
 {'label': 'LABEL_1', 'score': 0.9991251826286316},
 {'label': 'LABEL_0', 'score': 0.9966831803321838},
 {'label': 'LABEL_2', 'score': 0.6872998476028442},
 {'label': 'LABEL_1', 'score': 0.995182454586029},
 {'label': 'LABEL_2', 'score': 0.9958404898643494},
 {'label': 'LABEL_1', 'score': 0.9837796092033386},
 {'label': 'LABEL_2', 'score': 0.9983506202697754},
 {'label': 'LABEL_1', 'score': 0.9991958737373352},
 {'label': 'LABEL_1', 'score': 0.9991486072540283},
 {'label': 'LABEL_1', 'score': 0.9993591904640198},
 {'label': 'LABEL_1', 'score': 0.9979532957077026},
 {'label': 'LABEL_1', 'score': 0.9979967474937439}]

In [ ]:
# Try using the latest DistilRoberta Financial Sentiment Model - https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis

from transformers import pipeline

distilroberta_model = pipeline("sentiment-analysis", model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

distilroberta_model(["I love this movie", "This movie sucks!"])

distilroberta_model(["Energy shares drag on Wall Street as crude prices fall", "S&P 500 headed for a 67% downturn?", "Foreign investors navigate turmoil in Chinese markets with new playbook", "Fed Liquidity Drain Spoils Virus-Surge-Inspired Stock Buying-Panic", "Earnings per share ( EPS ) amounted to a loss of EUR0 .05",
                 "Equity markets likely to shine as Rupee's depreciation slows", "Investors cheered the blockbuster November jobs report", "Suven Life Sciences Q1 net up 16.68% at Rs 34.74 cr", "Airlines in sweet spot; expect SpiceJet to see rerating on IndiGo listing", "Twitter stock price target raised to $36.50 from $34.50 at Wedbush",
                 "A company is based in the district of the city", "The company did not disclose the price of the acquisition", "The 5 Best Finance Stocks of the Decade", "WeWork names new executives, path to profitability by 2023", "Chipotle highlights fast drive-thru lanes"])

config.json:   0%|          | 0.00/933 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

[{'label': 'negative', 'score': 0.9983526468276978},
 {'label': 'negative', 'score': 0.9978590607643127},
 {'label': 'neutral', 'score': 0.9973788261413574},
 {'label': 'negative', 'score': 0.9873884916305542},
 {'label': 'negative', 'score': 0.9981669187545776},
 {'label': 'positive', 'score': 0.9987959861755371},
 {'label': 'positive', 'score': 0.999421238899231},
 {'label': 'positive', 'score': 0.9997286200523376},
 {'label': 'positive', 'score': 0.9993207454681396},
 {'label': 'positive', 'score': 0.9993876218795776},
 {'label': 'neutral', 'score': 0.9998924732208252},
 {'label': 'neutral', 'score': 0.9998958110809326},
 {'label': 'neutral', 'score': 0.9998511075973511},
 {'label': 'neutral', 'score': 0.995762825012207},
 {'label': 'neutral', 'score': 0.9998747110366821}]

# 5. Fine-Tune KNN Classifier with data - Non-Huggingface

In [ ]:
# KNN Model
# Self Model

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MaxAbsScaler # Use MaxAbsScaler for sparse input
from sklearn.feature_extraction.text import TfidfVectorizer # Import TF-IDF Vectorizer
from sklearn.metrics import f1_score

X_train = small_train_dataset['text']
y_train = small_train_dataset['label']

X_test = small_test_dataset['text']
y_test = small_test_dataset['label']

knn_pipe = Pipeline([('tfidf', TfidfVectorizer()), # Transform text to numerical features
                     ('mms', MaxAbsScaler()),
                     ('knn', KNeighborsClassifier())])

params = [{'knn__n_neighbors': [5,10,15,20,25],
         'knn__weights': ['uniform', 'distance'],
         'knn__leaf_size': [1,2,3,4,5]}]

gs_knn = GridSearchCV(knn_pipe,
                      param_grid=params,
                      scoring='accuracy',
                      cv=5)

gs_knn.fit(X_train, y_train)

print("Best Hyperparameters: ")
print(gs_knn.best_params_)

print("Score on Training Data: ")
print(gs_knn.score(X_train, y_train))

print("pred_labels: ")
pred_labels = gs_knn.predict(X_test)
print(pred_labels)

f1 = f1_score(y_test, pred_labels, average='macro')
print(f"F1 score with 'macro'-averaging is {f1.round(5)}")

Best Hyperparameters: 
{'knn__leaf_size': 1, 'knn__n_neighbors': 15, 'knn__weights': 'uniform'}
Score on Training Data: 
0.575
pred_labels: 
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 2 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1]
F1 score with 'macro'-averaging is 0.30227


In [ ]:
# Test with Hyperparameters
knn = Pipeline([('tfidf', TfidfVectorizer()), # Transform text to numerical features
                ('mms', MaxAbsScaler()),
                ('knn', KNeighborsClassifier(n_neighbors=5, weights='distance', leaf_size=1))])

knn.fit(X_train, y_train)

print("Score on Training Data: ")
print(knn.score(X_train, y_train))

print("pred_labels: ")
pred_labels = knn.predict(X_test)
print(pred_labels)

f1 = f1_score(y_test, pred_labels, average='macro')
print(f"F1 score with 'macro'-averaging is {f1.round(5)}")


### On previous run, seems data-dependent, did better with '20' neighbors (0.565) rather than the suggested '10' neighbors (0.479), the predicted best was using '10' neighbors ###

Score on Training Data: 
1.0
pred_labels: 
[2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1
 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1]
F1 score with 'macro'-averaging is 0.27548


# 6. Fine-Tune Financial Sentiment Model 1500 - Huggingface

In [ ]:
# Fine-Tune a Pre-Trained Model - https://huggingface.co/docs/transformers/en/training
# financial-sentiment-model-1500-samples Model

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

import numpy as np
import evaluate

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("kevinwlip/financial-sentiment-model-1500-samples")
model = AutoModelForSequenceClassification.from_pretrained("kevinwlip/financial-sentiment-model-1500-samples")

# Tokenize the input data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True) # Tokenize the 'text' column

# Apply the preprocessing to the datasets
tokenized_train_dataset = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = small_test_dataset.map(preprocess_function, batched=True)

# Training arguments
repo_name = "kevinwlip/financial-sentiment-model-1500-samples-fine-tune"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
)

# Metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset, # Use the tokenized training dataset
    eval_dataset=tokenized_test_dataset, # Use the tokenized test dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=150, training_loss=0.45816914876302084, metrics={'train_runtime': 21.6464, 'train_samples_per_second': 110.873, 'train_steps_per_second': 6.93, 'total_flos': 30484603755648.0, 'train_loss': 0.45816914876302084, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.4706071615219116,
 'eval_accuracy': 0.8166666666666667,
 'eval_runtime': 0.576,
 'eval_samples_per_second': 520.863,
 'eval_steps_per_second': 32.988,
 'epoch': 2.0}

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

events.out.tfevents.1719127891.9bdd476daf66.3973.2:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1719127915.9bdd476daf66.3973.3:   0%|          | 0.00/411 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kevinwlip/financial-sentiment-model-1500-samples-fine-tune/commit/d5631f5a8dd00e0313f48f100e2561e70cb5a1db', commit_message='End of training', commit_description='', oid='d5631f5a8dd00e0313f48f100e2561e70cb5a1db', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

sentiment_model_tuned = pipeline("sentiment-analysis", model="kevinwlip/financial-sentiment-model-1500-samples-fine-tune")

""" Trying Text Data
sentiment_model_tuned(["I love this movie", "This movie sucks!"])

sentiment_model_tuned(["Energy shares drag on Wall Street as crude prices fall", "S&P 500 headed for a 67% downturn?", "Foreign investors navigate turmoil in Chinese markets with new playbook", "Fed Liquidity Drain Spoils Virus-Surge-Inspired Stock Buying-Panic", "Earnings per share ( EPS ) amounted to a loss of EUR0 .05",
                 "Equity markets likely to shine as Rupee's depreciation slows", "Investors cheered the blockbuster November jobs report", "Suven Life Sciences Q1 net up 16.68% at Rs 34.74 cr", "Airlines in sweet spot; expect SpiceJet to see rerating on IndiGo listing", "Twitter stock price target raised to $36.50 from $34.50 at Wedbush",
                 "A company is based in the district of the city", "The company did not disclose the price of the acquisition", "The 5 Best Finance Stocks of the Decade", "WeWork names new executives, path to profitability by 2023", "Chipotle highlights fast drive-thru lanes"])
"""

[{'label': 'LABEL_0', 'score': 0.8535113334655762},
 {'label': 'LABEL_0', 'score': 0.8078902959823608},
 {'label': 'LABEL_0', 'score': 0.5515747666358948},
 {'label': 'LABEL_0', 'score': 0.804631233215332},
 {'label': 'LABEL_0', 'score': 0.7821429371833801},
 {'label': 'LABEL_0', 'score': 0.8387794494628906},
 {'label': 'LABEL_1', 'score': 0.49390390515327454},
 {'label': 'LABEL_2', 'score': 0.9445022344589233},
 {'label': 'LABEL_1', 'score': 0.8420342803001404},
 {'label': 'LABEL_2', 'score': 0.904036283493042},
 {'label': 'LABEL_1', 'score': 0.9212526679039001},
 {'label': 'LABEL_1', 'score': 0.927251398563385},
 {'label': 'LABEL_1', 'score': 0.8745832443237305},
 {'label': 'LABEL_1', 'score': 0.9128180742263794},
 {'label': 'LABEL_1', 'score': 0.8809201717376709}]

# . Fine-Tune Pre-Existing ProsusAI Finbert Sentiment Model - Huggingface


In [ ]:
# ProsusAI Finbert Model

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

import numpy as np
import evaluate

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

# Tokenize the input data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True) # Tokenize the 'text' column

# Apply the preprocessing to the datasets
tokenized_train_dataset = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = small_test_dataset.map(preprocess_function, batched=True)

# Training arguments
repo_name = "kevinwlip/ProsusAI-finbert-1500-samples-fine-tune"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
)

# Metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset, # Use the tokenized training dataset
    eval_dataset=tokenized_test_dataset, # Use the tokenized test dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=150, training_loss=0.8426956176757813, metrics={'train_runtime': 70.9903, 'train_samples_per_second': 33.807, 'train_steps_per_second': 2.113, 'total_flos': 60548975254656.0, 'train_loss': 0.8426956176757813, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.6432344317436218,
 'eval_accuracy': 0.74,
 'eval_runtime': 0.9635,
 'eval_samples_per_second': 311.374,
 'eval_steps_per_second': 19.72,
 'epoch': 2.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1719128016.9bdd476daf66.3973.5:   0%|          | 0.00/411 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kevinwlip/ProsusAI-finbert-1500-samples-fine-tune/commit/8552a277f49ee705d5247a84b9a3228aee583e81', commit_message='End of training', commit_description='', oid='8552a277f49ee705d5247a84b9a3228aee583e81', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

finbert_model_tuned = pipeline("sentiment-analysis", model="kevinwlip/ProsusAI-finbert-1500-samples-fine-tune")

""" Trying Text Data
finbert_model_tuned(["I love this movie", "This movie sucks!"])

finbert_model_tuned(["Energy shares drag on Wall Street as crude prices fall", "S&P 500 headed for a 67% downturn?", "Foreign investors navigate turmoil in Chinese markets with new playbook", "Fed Liquidity Drain Spoils Virus-Surge-Inspired Stock Buying-Panic", "Earnings per share ( EPS ) amounted to a loss of EUR0 .05",
                 "Equity markets likely to shine as Rupee's depreciation slows", "Investors cheered the blockbuster November jobs report", "Suven Life Sciences Q1 net up 16.68% at Rs 34.74 cr", "Airlines in sweet spot; expect SpiceJet to see rerating on IndiGo listing", "Twitter stock price target raised to $36.50 from $34.50 at Wedbush",
                 "A company is based in the district of the city", "The company did not disclose the price of the acquisition", "The 5 Best Finance Stocks of the Decade", "WeWork names new executives, path to profitability by 2023", "Chipotle highlights fast drive-thru lanes"])
"""

[{'label': 'positive', 'score': 0.6072418093681335},
 {'label': 'negative', 'score': 0.5629236698150635},
 {'label': 'negative', 'score': 0.4589806795120239},
 {'label': 'positive', 'score': 0.4000363051891327},
 {'label': 'neutral', 'score': 0.7018522024154663},
 {'label': 'positive', 'score': 0.5568912029266357},
 {'label': 'neutral', 'score': 0.4511057436466217},
 {'label': 'neutral', 'score': 0.7547417283058167},
 {'label': 'negative', 'score': 0.6178648471832275},
 {'label': 'neutral', 'score': 0.7600321173667908},
 {'label': 'negative', 'score': 0.9120771288871765},
 {'label': 'negative', 'score': 0.8678937554359436},
 {'label': 'negative', 'score': 0.8215720653533936},
 {'label': 'negative', 'score': 0.8861539363861084},
 {'label': 'negative', 'score': 0.675176739692688}]

# 8. Fine-Tune Pre-Existing DistilRoberta Financial Sentiment Model - Huggingface



In [ ]:
# DistilRoberta Financial Sentiment Model

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer

import numpy as np
import evaluate

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

# Tokenize the input data
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True) # Tokenize the 'text' column

# Apply the preprocessing to the datasets
tokenized_train_dataset = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = small_test_dataset.map(preprocess_function, batched=True)

# Training arguments
repo_name = "kevinwlip/distilroberta-financial-sentiment-model-1500-samples-fine-tune"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",
    push_to_hub=True,
)

# Metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_train_dataset, # Use the tokenized training dataset
    eval_dataset=tokenized_test_dataset, # Use the tokenized test dataset
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Map:   0%|          | 0/1200 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=150, training_loss=0.559152577718099, metrics={'train_runtime': 58.657, 'train_samples_per_second': 40.916, 'train_steps_per_second': 2.557, 'total_flos': 30306597514272.0, 'train_loss': 0.559152577718099, 'epoch': 2.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3618961572647095,
 'eval_accuracy': 0.8566666666666667,
 'eval_runtime': 0.5772,
 'eval_samples_per_second': 519.718,
 'eval_steps_per_second': 32.915,
 'epoch': 2.0}

In [ ]:
trainer.push_to_hub()

events.out.tfevents.1719128095.9bdd476daf66.3973.7:   0%|          | 0.00/411 [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1719128036.9bdd476daf66.3973.6:   0%|          | 0.00/5.66k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/328M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/kevinwlip/distilroberta-financial-sentiment-model-1500-samples-fine-tune/commit/3f5646284edc9ce8aac31485a0057d606967003f', commit_message='End of training', commit_description='', oid='3f5646284edc9ce8aac31485a0057d606967003f', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
from transformers import pipeline

distilroberta_model_tuned = pipeline("sentiment-analysis", model="kevinwlip/distilroberta-financial-sentiment-model-1500-samples-fine-tune")

""" Trying Text Data
distilroberta_model_tuned(["I love this movie", "This movie sucks!"])

distilroberta_model_tuned(["Energy shares drag on Wall Street as crude prices fall", "S&P 500 headed for a 67% downturn?", "Foreign investors navigate turmoil in Chinese markets with new playbook", "Fed Liquidity Drain Spoils Virus-Surge-Inspired Stock Buying-Panic", "Earnings per share ( EPS ) amounted to a loss of EUR0 .05",
                 "Equity markets likely to shine as Rupee's depreciation slows", "Investors cheered the blockbuster November jobs report", "Suven Life Sciences Q1 net up 16.68% at Rs 34.74 cr", "Airlines in sweet spot; expect SpiceJet to see rerating on IndiGo listing", "Twitter stock price target raised to $36.50 from $34.50 at Wedbush",
                 "A company is based in the district of the city", "The company did not disclose the price of the acquisition", "The 5 Best Finance Stocks of the Decade", "WeWork names new executives, path to profitability by 2023", "Chipotle highlights fast drive-thru lanes"])
"""

[{'label': 'negative', 'score': 0.9644477963447571},
 {'label': 'negative', 'score': 0.8744779229164124},
 {'label': 'neutral', 'score': 0.8852159380912781},
 {'label': 'negative', 'score': 0.4812086522579193},
 {'label': 'negative', 'score': 0.9352914094924927},
 {'label': 'positive', 'score': 0.6802715063095093},
 {'label': 'positive', 'score': 0.9454445242881775},
 {'label': 'positive', 'score': 0.9886981248855591},
 {'label': 'positive', 'score': 0.9217618703842163},
 {'label': 'positive', 'score': 0.9754182696342468},
 {'label': 'neutral', 'score': 0.9931836724281311},
 {'label': 'neutral', 'score': 0.9946097135543823},
 {'label': 'neutral', 'score': 0.9449321627616882},
 {'label': 'neutral', 'score': 0.5704339742660522},
 {'label': 'neutral', 'score': 0.9486281871795654}]

# Analysis - Please click into here to read the data easier.

*   List item
*   List item



Starting from Step 4. I was analyzing new data with multiple models - Self Sentiment Model, Bert Model, Twitter Roberta Model, ProsusAI Finbert Model, Sigma's Financial Sentiment Analysis Model, and DistilRoberta Financial Sentiment Model.

I ran the same 5 Negative, 5 Positive, and 5 Neutral statements taken from my 1500 sample dataset and it seemed like ProsusAI Finbert and The DistilRoberta Financial Model performed better, both accurately predicting with at most 2 wrong on a few tries.

After fine tuning the three selected models for analysis, I noticed that two of the models' predictions improved (Self Sentiment Model and DistilRoberta Financial Sentiment Model), the exception being ProsusAI Finbert Model which did worse. With 'distilroberta_model_tuned' predicting on the labels perfectly on the 15 statements.

I believe I will use the 'distilroberta_mode' overall for my Capstone due to its accuracy overall.


sentiment_model - LABEL_0: Negative, LABEL_1: Positive, LABEL_2: Neutral
[{'label': 'LABEL_0', 'score': 0.47167298197746277},
 {'label': 'LABEL_0', 'score': 0.34854891896247864},
 {'label': 'LABEL_2', 'score': 0.4000025987625122},   X
 {'label': 'LABEL_0', 'score': 0.37904348969459534},
 {'label': 'LABEL_0', 'score': 0.3914853036403656},
 {'label': 'LABEL_0', 'score': 0.45039665699005127},  X
 {'label': 'LABEL_2', 'score': 0.6083877086639404},   X
 {'label': 'LABEL_1', 'score': 0.6673786640167236},
 {'label': 'LABEL_2', 'score': 0.5597862601280212},   X
 {'label': 'LABEL_1', 'score': 0.6164636611938477},
 {'label': 'LABEL_2', 'score': 0.606207549571991},
 {'label': 'LABEL_2', 'score': 0.7558310031890869},
 {'label': 'LABEL_2', 'score': 0.6832563877105713},
 {'label': 'LABEL_2', 'score': 0.6757391095161438},
 {'label': 'LABEL_2', 'score': 0.6516631245613098}]


sentiment_model_tuned - LABEL_0: Negative, LABEL_1: Positive, LABEL_2: Neutral
[{'label': 'LABEL_0', 'score': 0.9350849986076355},
 {'label': 'LABEL_0', 'score': 0.9179285764694214},
 {'label': 'LABEL_0', 'score': 0.8534387946128845},
 {'label': 'LABEL_0', 'score': 0.9133590459823608},
 {'label': 'LABEL_0', 'score': 0.9111049771308899},
 {'label': 'LABEL_0', 'score': 0.9243319034576416},   X
 {'label': 'LABEL_2', 'score': 0.5825300216674805},   X
 {'label': 'LABEL_1', 'score': 0.922419548034668},
 {'label': 'LABEL_1', 'score': 0.6049041748046875},
 {'label': 'LABEL_1', 'score': 0.8833795785903931},
 {'label': 'LABEL_2', 'score': 0.741992175579071},
 {'label': 'LABEL_2', 'score': 0.8895606994628906},
 {'label': 'LABEL_2', 'score': 0.8496867418289185},
 {'label': 'LABEL_2', 'score': 0.7117111682891846},
 {'label': 'LABEL_2', 'score': 0.7510005831718445}]


 finbert_model
[{'label': 'negative', 'score': 0.9179691076278687},
 {'label': 'negative', 'score': 0.9538722038269043},
 {'label': 'neutral', 'score': 0.8057689070701599},   X
 {'label': 'negative', 'score': 0.5092411637306213},
 {'label': 'negative', 'score': 0.7875530123710632},
 {'label': 'negative', 'score': 0.5851684808731079},  X
 {'label': 'positive', 'score': 0.6592949628829956},
 {'label': 'positive', 'score': 0.9493573307991028},
 {'label': 'positive', 'score': 0.8135282397270203},
 {'label': 'positive', 'score': 0.9495117664337158},
 {'label': 'neutral', 'score': 0.9443269968032837},
 {'label': 'neutral', 'score': 0.9468695521354675},
 {'label': 'neutral', 'score': 0.6568445563316345},
 {'label': 'neutral', 'score': 0.7671145796775818},
 {'label': 'neutral', 'score': 0.9020786285400391}]


finbert_model_tuned
[{'label': 'positive', 'score': 0.6108507513999939},  X
 {'label': 'negative', 'score': 0.4695791006088257},
 {'label': 'positive', 'score': 0.47678446769714355}, X
 {'label': 'positive', 'score': 0.47963422536849976}, X
 {'label': 'positive', 'score': 0.7220267057418823},  X
 {'label': 'positive', 'score': 0.5846428871154785},
 {'label': 'positive', 'score': 0.45479774475097656},
 {'label': 'negative', 'score': 0.7035203576087952},  X
 {'label': 'negative', 'score': 0.4356476366519928},  X
 {'label': 'positive', 'score': 0.519198477268219},
 {'label': 'neutral', 'score': 0.893254816532135},
 {'label': 'neutral', 'score': 0.9592297673225403},
 {'label': 'neutral', 'score': 0.767744243144989},
 {'label': 'neutral', 'score': 0.9325315356254578},
 {'label': 'neutral', 'score': 0.5289221405982971}]


distilroberta_model
[{'label': 'negative', 'score': 0.9983526468276978},
 {'label': 'negative', 'score': 0.9978590607643127},
 {'label': 'neutral', 'score': 0.9973788261413574},   X
 {'label': 'negative', 'score': 0.9873886108398438},
 {'label': 'negative', 'score': 0.9981669187545776},
 {'label': 'positive', 'score': 0.9987959861755371},
 {'label': 'positive', 'score': 0.999421238899231},
 {'label': 'positive', 'score': 0.9997286200523376},
 {'label': 'positive', 'score': 0.9993207454681396},
 {'label': 'positive', 'score': 0.9993876218795776},
 {'label': 'neutral', 'score': 0.9998924732208252},
 {'label': 'neutral', 'score': 0.9998958110809326},
 {'label': 'neutral', 'score': 0.9998511075973511},
 {'label': 'neutral', 'score': 0.9957629442214966},
 {'label': 'neutral', 'score': 0.9998747110366821}]


distilroberta_model_tuned
[{'label': 'negative', 'score': 0.9701110124588013},
 {'label': 'negative', 'score': 0.969354510307312},
 {'label': 'negative', 'score': 0.5647386312484741},
 {'label': 'negative', 'score': 0.8065626621246338},
 {'label': 'negative', 'score': 0.9823943972587585},
 {'label': 'neutral', 'score': 0.8046205043792725},
 {'label': 'neutral', 'score': 0.7604220509529114},
 {'label': 'neutral', 'score': 0.9848699569702148},
 {'label': 'neutral', 'score': 0.9329875707626343},
 {'label': 'neutral', 'score': 0.975169837474823},
 {'label': 'positive', 'score': 0.8283220529556274},
 {'label': 'positive', 'score': 0.7942936420440674},
 {'label': 'positive', 'score': 0.8845605850219727},
 {'label': 'neutral', 'score': 0.5417729020118713},
 {'label': 'positive', 'score': 0.6406908631324768}]   ALL PASS


In [ ]:
# Add links to the models

# Use test/validation data on the models, establish baseline  Step 7


# Fine tune models, create models   Step 8
# Run evaluation on the fine tune models, compare models to the baseline --> Fine tune should be better than baseline
# Create graphs for the models

#Amal Comments for Step 7 + Finish  Step 8
#- Bert, roBerta, Other Financial Sentimental Analysis Model
#- Increase the total size (train + test) from 1500 (small) and 5000 (medium) even sampled Negative, Positive, Neutral
#- Use correct labels - [{'label': 'LABEL_1', 'score': 0.7213886976242065} fix labels for this Step 4
# {'label': 'LABEL_1', 'score': 0.6998449563980103}]
#- Pick 1 https://huggingface.co/models?other=financial-sentiment-analysis -- > https://huggingface.co/ProsusAI/finbert

#- Evaluation - Evaluate, Charts, Accuracy.
# Add Machine learning performance metric model define accuracy, F1, RSME, LOC